**Project: NBA Shooting Trends Analysis**

This project analyzes how shooting patterns in the NBA have changed over time. By leveraging the nba_api library, we retrieve player career statistics and use Python libraries for data manipulation and visualization to explore changes in shooting metrics across seasons.

In [2]:
import pandas as pd
import plotly.express as px
from pandasql import sqldf
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import warnings
import time

In [3]:
warnings.filterwarnings('ignore')

The player IDs are crucial for fetching detailed player statistics later in the process, as the ID is a mandatory parameter in the API player stats function.

In [4]:
p_get_players = players.get_players()

all_players = pd.DataFrame(p_get_players)
all_playersID = all_players["id"]
print(all_players)
print(all_playersID)

           id            full_name first_name     last_name  is_active
0       76001       Alaa Abdelnaby       Alaa     Abdelnaby      False
1       76002      Zaid Abdul-Aziz       Zaid    Abdul-Aziz      False
2       76003  Kareem Abdul-Jabbar     Kareem  Abdul-Jabbar      False
3          51   Mahmoud Abdul-Rauf    Mahmoud    Abdul-Rauf      False
4        1505    Tariq Abdul-Wahad      Tariq   Abdul-Wahad      False
...       ...                  ...        ...           ...        ...
5029  1627826          Ivica Zubac      Ivica         Zubac       True
5030    78650           Matt Zunic       Matt         Zunic      False
5031  1641783     Tristan da Silva    Tristan      da Silva       True
5032  1628427        Vlatko Čančar     Vlatko        Čančar       True
5033   203967          Dario Šarić      Dario         Šarić       True

[5034 rows x 5 columns]
0         76001
1         76002
2         76003
3            51
4          1505
         ...   
5029    1627826
5030      7

This part of the code collects the career statistics for all players in the all_playersID list. It fetches the data from the NBA API, combines each player's stats into a single DataFrame, and then saves this complete dataset to a CSV file.

This step allows for easy sharing, further analysis, or integration with other tools. Saving data to CSV also provides a backup of the analysis results that can be revisited later without needing to rerun the entire data collection and processing pipeline.

In [4]:
all_players_stats = pd.DataFrame()

for player in all_playersID:
    try:
        player_stats = playercareerstats.PlayerCareerStats(player, per_mode36="Totals").get_data_frames()[0]
        all_players_stats = pd.concat([all_players_stats, player_stats])
        
        # Sleep (timeout)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error fetching data for player {player}: {e}")

all_players_stats.to_csv("nba.csv")

In [5]:
dfstats = pd.read_csv("nba.csv")


**Using SQL for Data Aggregation:**

Count of Players Attempting 100+ Three-Pointers per season.

Total Three-Point Attempts (3GA) and Total Field Goal Attempts (FGA) per season.

Count of Players Playing 100+ Minutes per season.

In [6]:
stats_3ptseason = sqldf("SELECT d.season_id ,COUNT(d.PLAYER_ID) as PLAYERS_3PT FROM dfstats d WHERE d.FG3A >= 100  GROUP BY d.SEASON_ID ORDER BY SEASON_ID desc LIMIT 25", locals())
stats_season = sqldf("SELECT d.season_id, SUM(d.FGA) as FGA, SUM(d.FG3A) as FG3A FROM dfstats d GROUP BY d.SEASON_ID ORDER BY SEASON_ID desc LIMIT 25", locals())
stats_player = sqldf("SELECT d.season_id ,COUNT(d.PLAYER_ID) as PLAYERS FROM dfstats d WHERE d.min >= 100 GROUP BY d.SEASON_ID ORDER BY SEASON_ID desc LIMIT 25", locals())

stats = stats_season.merge(stats_3ptseason, on="SEASON_ID", how="left").merge(stats_player, on="SEASON_ID", how="left")

stats["PLAYERS_3PT"] = stats["PLAYERS_3PT"].fillna(0).astype(int)
stats["FG3A"] = stats["FG3A"].fillna(0).astype(int)

stats["PCT PLAYERS 3PT"] = ((stats["PLAYERS_3PT"]/stats["PLAYERS"])*100).astype(int)
stats["PCT FG3A"] = ((stats["FG3A"]/stats["FGA"])*100).astype(int)

stats["SEASON_ID"] = stats["SEASON_ID"].str[:4]
stats["PCT PLAYERS 3PT STR"] = stats["PCT PLAYERS 3PT"].astype(str)+"%"
stats["PCT FG3A STR"] = stats["PCT FG3A"].astype(str)+"%"
stats = stats[::-1]
print(stats)

   SEASON_ID     FGA    FG3A  PLAYERS_3PT  PLAYERS  PCT PLAYERS 3PT  PCT FG3A  \
24      2000  206292   34290          128      474               27        16   
23      2001  205985   37517          133      443               30        18   
22      2002  203368   37101          142      422               33        18   
21      2003  215004   39708          141      494               28        18   
20      2004  220073   43906          159      515               30        19   
19      2005  210486   42898          153      499               30        20   
18      2006  207465   44269          167      458               36        21   
17      2007  219275   48484          169      513               32        22   
16      2008  221478   50535          178      502               35        22   
15      2009  221161   49184          182      499               36        22   
14      2010  230212   52915          202      547               36        22   
13      2011  170292   38277

Rise in Three-Point Shooting as a Necessity:

This project visualizes the increase in three-point attempt percentage over time, especially after the 2014 season, showing how the game has evolved toward more three-point shooting.

In [7]:
fig = px.histogram(stats, x="SEASON_ID", y= "PCT FG3A", nbins= 25)

fig.update_traces(text=stats["PCT FG3A STR"], textposition = "auto", marker_color = "#4C78A8")
fig.update_layout(
    title = "Percentage of 3-Point Attempts Relative to Total Field Goal Attempts by Season",
    xaxis_title = "Season",
    yaxis_title = "Percentage of 3PT Field Goal"
)
fig.add_annotation(
    x=15, y= 31,
    text="Best Regular Season in History<br>GSW 73-9<br>Curry First Ever Unanimous MVP",
    arrowhead=2,
    ax=0,
    ay=-40
)


fig.show()

Rise in Three-Point Shooting as a Necessity:

This visualization illustrates the shift in the NBA where it has become almost essential for players to be able to shoot three-pointers. In the early 2000s, only a small percentage of players attempted more than 100 three-pointers per season. However, as of recent seasons, this proportion has significantly increased, showing how the game has evolved toward a higher reliance on three-point shooting.

In [8]:
fig = px.histogram(stats, x="SEASON_ID", y= "PCT PLAYERS 3PT", nbins= 25)
fig.update_traces(text=stats["PCT PLAYERS 3PT STR"], textposition = "auto", marker_color = "#4C78A8")

fig.update_layout(
    title = "Percentage of Players with 100+ 3-Point Attempts by Season <br><sup> The analysis took into account only those players who participated for 100 or more minutes in the season.",
    xaxis_title = "Season",
    yaxis_title = "Percentage of Players"
)

fig.add_shape(
    type="line",
    x0=14,  # Start of the x-axis range
    x1=24,  # End of the x-axis range
    y0=42,  # Y-value for the line
    y1=42,  # Same Y-value to make it horizontal
    line=dict(color="Light Blue", width=1.3, dash="dash"),  # Customize line style
)

fig.add_annotation(
    x=11, y= 35,
    text="1/3 of Players Shot 100+ 3 Pointer",
    arrowhead=2,
    ax=0,
    ay=-40
)
fig.add_annotation(
    x=18, y= 53,
    text="1/2 of Players Shot 100+ 3 Pointer",
    arrowhead=2,
    ax=0,
    ay=-40
)
fig.add_annotation(
    x=24, y= 55,
    text="40% of the Season Completed",
    arrowhead=2,
    ax=0,
    ay=-65
)

fig.show()
